In [ ]:
#Writing Replacement Function
import pandas as pd
import numpy as np
from utils.Dataframe_tools import PSD 
from utils import iterateSubDirFiles

#Setting the Values
IO='input files\KPBom.xlsx'

find_remove=[{
    '91840054':'99717392',
    '91840056':'99717485',
    '98441969':'92758523',
    '96698939':'92758528',     # in KPbom.xlsx 
    '96698968':'92758121',     # in KPbom.xlsx
    '96768950':'92758767',     # in KPbom.xlsx
    '99413027':'92758120',
    '1020-3/4':'1020-5/6',
    '1020-3_4':'1020-5_6',
    'Desc-1020-3_4-KP_4':'Desc-1020-5_6-KP_6',
    'Desc-1020-3_4-KP_3':'Desc-1020-5_6-KP_5'
},]

def find_and_replace(df:pd.DataFrame,to_replace,new_value)->None:
    df.replace(to_replace=to_replace,value=new_value,regex=True,inplace=True)

def change_psd_file(filename,function,*args,**kwargs)->list:
    sheet_list=pd.ExcelFile(filename).sheet_names
    df_dict={}
    for sheet in sheet_list:
        try:
            df=PSD(io=filename,sheet_name=sheet)
            function(df.psd_data,*args,**kwargs)
            df_dict.update({sheet:df})
        except TypeError:
            print(sheet.title()+" is not in PSD format.")
    return df_dict
        

dfs=change_psd_file(filename=IO,function=find_and_replace,to_replace=list(find_remove[0].keys()),new_value=list(find_remove[0].values()))



In [ ]:
case=dfs['Case']
data=case.psd_data
header=case.header_data

In [60]:


# for id in IDs:
#     data2.get_group(id)

endr=header[header.iloc[:,0].str.startswith("[")].index[0]

classes=header[:endr].values.tolist()
types=header.iloc[endr].tolist()
arr=[]
#Now I need to iterate over the classes
for row in classes:
    dct={}
    new_data=data
    new_data.columns=row
    column_numbers = [x for x in range(new_data.shape[1])]
    for j,col in enumerate(row):
        dct2={}
        if j==0:
            dct.update({'class':col})
        if str(col).strip().lower()=='nan':
            column_numbers .remove(j)
        else:
            if j==0:
                column_numbers.remove(0)
            else:
                dct2.update({col:types[j]})
        if str(col).strip().lower()=='id':
            data2=data.groupby(col)
            IDs=list(data2.groups.keys())
            dct.update({'instances':IDs})
    new_data=new_data.iloc[:, column_numbers]
    dct.update({'data':new_data})
    dct.update({'DataTypes':dct2})
    arr.append(dct)

    


In [55]:
XML=arr[0]['data'].to_xml(index=False,row_name='instance',root_name='class')
a=ET.fromstring(XML)
ET.SubElement(a,"Instances")
b=ET.ElementTree(a)
b.write('test.xml')

In [46]:
#Converting to XML

import xml.etree.ElementTree as ET

root=ET.Element('FirepondSPCDatabase',attrib={'version':'3.0'}) #Creates the Root Node

with open('test.xml','wb') as doc:
    doc.write(ET.tostring(root,encoding='UTF-8',xml_declaration=True,short_empty_elements=False))



AttributeError: module 'xml.etree.ElementTree' has no attribute 'write'

In [ ]:
new_data.iloc[:,[0,1]]